In [44]:
import numpy as np
import pandas as pd
import os
import cv2
import datetime
import pandas as pd
import random
from matplotlib import pyplot as plt

from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Conv2DTranspose, Reshape,  Activation, Lambda
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

In [45]:
import tensorflow as tf
print("GPUs detectadas:", len(tf.config.list_physical_devices('GPU')))
tf.config.list_physical_devices('GPU')

GPUs detectadas: 1


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [46]:
def plot_losses(history):
    plt.rcParams['figure.figsize'] = [20, 5]
    f, (ax1, ax2) = plt.subplots(1, 2, sharex=True)

    ax1.set_title('Losses')
    ax1.set_xlabel('epoch')
    ax1.legend(loc="upper right")
    ax1.grid()
    ax1.plot(history['loss'], label='Training loss')
    ax1.plot(history['val_loss'], label='Validation loss')
    ax1.legend()

    plt.show()


def plot_resultados(model, carpeta, height=64, width=64, n=4):
    """
    Muestra comparaciones entre imágenes originales y reconstruidas por el autoencoder.
    """
    # Seleccionar imágenes aleatorias
    archivos = os.listdir(carpeta)
    archivos_img = random.sample(archivos, n)

    # Cargar y normalizar las imágenes
    imgs_originales = []
    for nombre in archivos_img:
        img = load_img(os.path.join(carpeta, nombre), target_size=(height, width))
        img_array = img_to_array(img) / 255.0  # normalizar a [0,1]
        imgs_originales.append(img_array)

    imgs_originales = np.array(imgs_originales)

    # Reconstruir con el modelo
    imgs_reconstruidas = model.predict(imgs_originales)

    # Mostrar resultados
    plt.figure(figsize=(12, 6))
    for i in range(n):
        # Imagen original
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(imgs_originales[i])
        ax.set_title("Original")
        ax.axis("off")

        # Imagen reconstruida
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(imgs_reconstruidas[i])
        ax.set_title("Reconstruida")
        ax.axis("off")

    plt.tight_layout()
    plt.show()

## Defining Hiperparameters

In [47]:
import tensorflow as tf
from tensorflow.keras import layers

# Parámetros 
img_height = 64
img_width = 64
img_channels = 3
input_shape = (img_height, img_width, img_channels)
latent_dim = 512
batch_size = 256
image_dir = "/kaggle/input/animefacedataset"

In [48]:
from tensorflow.keras.utils import image_dataset_from_directory

# Dataset de entrenamiento y validación con split
train_ds = image_dataset_from_directory(
    image_dir,
    label_mode=None,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=44
)

val_ds = image_dataset_from_directory(
    image_dir,
    label_mode=None,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=44
)

# Normalizamos
def preprocess(img):
    img = (tf.cast(img, tf.float32) / 127.5) - 1.0
    return img, img

train_ds = train_ds.map(preprocess)
val_ds = val_ds.map(preprocess)

# Prefetch para rendimiento
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

Found 63565 files.
Using 50852 files for training.
Found 63565 files.
Using 12713 files for validation.


### VAE

In [49]:
input_img = Input(shape=input_shape)

# Encoder

x_enc = Conv2D(16, (3, 3), strides=2, padding="same")(input_img)
x_enc = BatchNormalization()(x_enc)
x_enc = Activation("relu")(x_enc)

x_enc = Conv2D(32, (3, 3), strides=2, padding="same")(x_enc)
x_enc = BatchNormalization()(x_enc)
x_enc = Activation("relu")(x_enc)

x_enc = Conv2D(64, (3, 3), strides=2, padding="same")(x_enc)
x_enc = BatchNormalization()(x_enc)
x_enc = Activation("relu")(x_enc)

x_enc = Conv2D(128, (3, 3), strides=2, padding="same")(x_enc)
x_enc = BatchNormalization()(x_enc)
x_enc = Activation("relu")(x_enc)

x_enc = Conv2D(256, (3, 3), strides=2, padding="same")(x_enc)
x_enc = BatchNormalization()(x_enc)
x_enc = Activation("relu")(x_enc)

x_enc = Flatten()(x_enc)
x_enc = Dense(512, activation="relu")(x_enc)

# VAE: media y log-varianza
mu = Dense(latent_dim, name="mu")(x_enc)
log_var = Dense(latent_dim, name="log_var")(x_enc)

# Muestreo
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.random.normal(shape=tf.shape(z_mean))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling, name="z")([mu, log_var])

encoder = Model(input_img, [mu, log_var, z], name="encoder")

In [50]:
encoder.summary()

Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 64, 64, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_10 (Conv2D)  │ (None, 32, 32,    │        448 │ input_layer_6[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │         64 │ conv2d_10[0][0]   │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_15       │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_11 (Conv2D)  │ (None, 16, 16,    │      4,640 │ activation_15[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 16,    │        128 │ conv2d_11[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_16       │ (None, 16, 16,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_12 (Conv2D)  │ (None, 8, 8, 64)  │     18,496 │ activation_16[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 8, 8, 64)  │        256 │ conv2d_12[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_17       │ (None, 8, 8, 64)  │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_13 (Conv2D)  │ (None, 4, 4, 128) │     73,856 │ activation_17[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 4, 4, 128) │        512 │ conv2d_13[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_18       │ (None, 4, 4, 128) │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_14 (Conv2D)  │ (None, 2, 2, 256) │    295,168 │ activation_18[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 2, 2, 256) │      1,024 │ conv2d_14[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_19       │ (None, 2, 2, 256) │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 1024)      │          0 │ activation_19[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 512)       │    524,800 │ flatten_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 1,444,704 (5.51 MB)

 Trainable params: 1,443,712 (5.51 MB)

 Non-trainable params: 992 (3.88 KB)

In [51]:
# Decoder
input_lat = Input(shape=(latent_dim,))

x_dec = Dense(2*2* 512, activation="relu")(input_lat)
x_dec = Reshape((2, 2, 512))(x_dec)

x_dec = Conv2DTranspose(256, (3, 3), strides=2, padding="same")(x_dec)
x_dec = BatchNormalization()(x_dec)
x_dec = Activation("relu")(x_dec)

x_dec = Conv2DTranspose(128, (3, 3), strides=2, padding="same")(x_dec)
x_dec = BatchNormalization()(x_dec)
x_dec = Activation("relu")(x_dec)

x_dec = Conv2DTranspose(64, (3, 3), strides=2, padding="same")(x_dec)
x_dec = BatchNormalization()(x_dec)
x_dec = Activation("relu")(x_dec)

x_dec = Conv2DTranspose(32, (3, 3), strides=2, padding="same")(x_dec)
x_dec = BatchNormalization()(x_dec)
x_dec = Activation("relu")(x_dec)

x_dec = Conv2DTranspose(16, (3, 3), strides=2, padding="same")(x_dec)
x_dec = BatchNormalization()(x_dec)
x_dec = Activation("relu")(x_dec)

decoded = Conv2DTranspose(3, (3, 3), activation="sigmoid", padding="same")(x_dec)

decoder = Model(input_lat, decoded, name="decoder")


In [52]:
decoder.summary()

Model: "decoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 2048)           │     1,050,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_4 (Reshape)             │ (None, 2, 2, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_8              │ (None, 4, 4, 256)      │     1,179,904 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_20          │ (None, 4, 4, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_20 (Activation)      │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_9              │ (None, 8, 8, 128)      │       295,040 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_21          │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_21 (Activation)      │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_10             │ (None, 16, 16, 64)     │        73,792 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_22          │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_22 (Activation)      │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_11             │ (None, 32, 32, 32)     │        18,464 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_23          │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_23 (Activation)      │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_12             │ (None, 64, 64, 16)     │         4,624 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_24          │ (None, 64, 64, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_24 (Activation)      │ (None, 64, 64, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_13             │ (None, 64, 64, 3)      │           435 │
│ (Conv2DTranspose)               │                        │               │
└─────────────────────────────────┴────────────────────────┴─────────────

 Total params: 2,624,867 (10.01 MB)

 Trainable params: 2,623,875 (10.01 MB)

 Non-trainable params: 992 (3.88 KB)

In [58]:
class VAE(Model):
    def __init__(self, encoder, decoder, beta=1.0, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.beta = beta

        # métricas
        self.total_loss_tracker = tf.keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = tf.keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = tf.keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [self.total_loss_tracker,
                self.reconstruction_loss_tracker,
                self.kl_loss_tracker]

    def train_step(self, data):
        x,y= data
        
        with tf.GradientTape() as tape:
            mu, log_var, z = self.encoder(x, training=True)
            x_decoded = self.decoder(z, training=True)

            reconstruction_loss_fn = tf.keras.losses.MeanSquaredError()
            reconstruction_loss = reconstruction_loss_fn(y, x_decoded) * (img_height * img_width)

            kl_loss = -0.5 * (1 + log_var - tf.square(mu) - tf.exp(log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            
            total_loss = reconstruction_loss + self.beta * kl_loss

        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

        # actualizar métricas
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)

        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

vae = VAE(encoder, decoder)
vae.summary()

Model: "vae_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder (Functional)            │ ((None, 512), (None,   │     1,444,704 │
│                                 │ 512), (None, 512))     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder (Functional)            │ (None, 64, 64, 3)      │     2,624,867 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,069,571 (15.52 MB)

 Trainable params: 4,067,587 (15.52 MB)

 Non-trainable params: 1,984 (7.75 KB)

## Train

In [59]:
vae.compile(Adam(learning_rate=1e-3))

In [60]:
h = vae.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3
)

Epoch 1/3
199/199 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - kl_loss: 4.6880 - loss: 1193.9199 - reconstruction_loss: 1189.2319

NotImplementedError: Exception encountered when calling VAE.call().

[1mModel VAE does not have a `call()` method implemented.[0m

Arguments received by VAE.call():
  • args=('tf.Tensor(shape=(None, 64, 64, 3), dtype=float32)',)
  • kwargs=<class 'inspect._empty'>

In [ ]:
plot_losses(h.history)

In [ ]:
image_dir="/kaggle/input/animefacedataset/images"
plot_resultados(vae,image_dir)

In [ ]:
# Current timestamp
timestamp = datetime.datetime.now().strftime("%m_%d_%H:%M")
## Make sure everything saves correctly
os.makedirs("models", exist_ok=True)

model_path = f"models/vae_v2_{timestamp}.keras"

# Save the model
vae.save(model_path)
